<a href="https://colab.research.google.com/github/CursedOrca/Algorithms/blob/master/Titanic_classification_Final_Version_Edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [0]:
from google.colab import files
import io

# Prompting to upload the train.csv
uploaded_train = files.upload()


Saving train.csv to train (3).csv


In [0]:
df = pd.read_csv(io.BytesIO(uploaded_train['train.csv']), index_col='PassengerId')

In [0]:
# Prompting to upload the test.csv file
uploaded_test = files.upload()

Saving test.csv to test (5).csv


In [0]:
# Save the testing data into a dataframe
df_test = pd.read_csv(io.BytesIO(uploaded_test['test.csv']), index_col='PassengerId')

In [0]:
#Please check the test dataframe's columns
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [0]:
df.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [0]:
#Join
df["Family"] = df["SibSp"] + df["Parch"]
df.head()
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3


In [0]:
del df['SibSp'], df['Parch']

In [0]:
df

,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Family
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,211536,13.0000,NaN,S,0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,112053,30.0000,B42,S,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,W./C. 6607,23.4500,NaN,S,3


In [0]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
Ticket        0
Fare          0
Cabin       687
Embarked      2
Family        0
dtype: int64

In [0]:
del df['Cabin']


In [0]:
df

,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Family
PassengerId,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,211536,13.0000,S,0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,112053,30.0000,S,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,W./C. 6607,23.4500,S,3


In [0]:
#Avergae age
avg_age = df.groupby('Age')
df.mean()


Survived     0.383838
Pclass       2.308642
Age         29.699118
Fare        32.204208
Family       0.904602
dtype: float64

In [0]:
#Fill the missing value
#Age
df['Age'].value_counts(dropna=False)

NaN      177
24.00     30
22.00     27
18.00     26
28.00     25
        ... 
36.50      1
55.50      1
66.00      1
23.50      1
0.42       1
Name: Age, Length: 89, dtype: int64

In [0]:
df['Age'].fillna(value=29.699118, inplace=True)
df['Age'].value_counts(dropna=False)

29.699118    177
24.000000     30
22.000000     27
18.000000     26
28.000000     25
            ... 
36.500000      1
12.000000      1
14.500000      1
0.420000       1
0.670000       1
Name: Age, Length: 89, dtype: int64

In [0]:
#Embarked
df['Embarked'].fillna(value='S', inplace=True)
df['Embarked'].value_counts(dropna=False)

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [0]:
#Check how many missing value
df.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
Ticket      0
Fare        0
Embarked    0
Family      0
dtype: int64

In [0]:
#Replace the gender
df['Sex'] = df['Sex'].replace(to_replace ="male", 
                 value =0) 
df['Sex'] = df['Sex'].replace(to_replace ="female", 
                 value =1) 
df

,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Family
PassengerId,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",0,22.000000,A/5 21171,7.2500,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.000000,PC 17599,71.2833,C,1
3,1,3,"Heikkinen, Miss. Laina",1,26.000000,STON/O2. 3101282,7.9250,S,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.000000,113803,53.1000,S,1
5,0,3,"Allen, Mr. William Henry",0,35.000000,373450,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",0,27.000000,211536,13.0000,S,0
888,1,1,"Graham, Miss. Margaret Edith",1,19.000000,112053,30.0000,S,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,29.699118,W./C. 6607,23.4500,S,3


In [0]:
#Store variable
feature_cols = ['Pclass', 'Sex', 'Age', 'Fare', 'Family']
X = df.loc[:, feature_cols]
X.head()



,Pclass,Sex,Age,Fare,Family
PassengerId,,,,,
1,3,0,22.0,7.2500,1
2,1,1,38.0,71.2833,1
3,3,1,26.0,7.9250,0
4,1,1,35.0,53.1000,1
5,3,0,35.0,8.0500,0


In [0]:
#Store labels
Y = df.Survived
Y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [0]:
#Scale
from sklearn.preprocessing import scale
X_scaled = scale(X)

In [0]:
#Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2)
#Print the shape
X_train.shape

(712, 5)

In [0]:
#Create a model
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]

#add model layers
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [0]:
#Compile
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=["accuracy"])

In [0]:
# Fit the data
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=8)
from keras.utils import to_categorical

model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

Train on 569 samples, validate on 143 samples
Epoch 1/30
569/569 [==============================] - 0s 591us/step - loss: 0.2926 - acc: 0.6731 - val_loss: 0.2460 - val_acc: 0.6923
Epoch 2/30
569/569 [==============================] - 0s 44us/step - loss: 0.2072 - acc: 0.7434 - val_loss: 0.1995 - val_acc: 0.7273
Epoch 3/30
569/569 [==============================] - 0s 49us/step - loss: 0.1791 - acc: 0.7750 - val_loss: 0.1785 - val_acc: 0.7622
Epoch 4/30
569/569 [==============================] - 0s 49us/step - loss: 0.1670 - acc: 0.7821 - val_loss: 0.1688 - val_acc: 0.7692
Epoch 5/30
569/569 [==============================] - 0s 53us/step - loss: 0.1596 - acc: 0.7944 - val_loss: 0.1629 - val_acc: 0.7832
Epoch 6/30
569/569 [==============================] - 0s 53us/step - loss: 0.1553 - acc: 0.7909 - val_loss: 0.1562 - val_acc: 0.7902
Epoch 7/30
569/569 [==============================] - 0s 46us/step - loss: 0.1513 - acc: 0.7944 - val_loss: 0.1526 - val_acc: 0.8042
Epoch 8/30
569/569 [==

In [0]:
#test score and accuarcy
test_loss, test_acc = model.evaluate(X_test, y_test)

179/179 [==============================] - 0s 54us/step


In [0]:
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 10)                60        
_________________________________________________________________
dense_32 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
print(test_loss)

0.13195131877281147


In [0]:
print(test_acc)

0.8324022329719373


In [0]:
# 29

neural_network = Sequential()
n_cols_2 = X_train.shape[1]
neural_network.add(Dense(250, activation='relu', input_shape=(n_cols_2,)))
neural_network.add(Dense(250, activation='relu'))
neural_network.add(Dense(250, activation='relu'))
neural_network.add(Dense(10, activation='tanh'))

#Compile
neural_network.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

In [0]:
#Fit the data
# neural_network.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

### Classify the features

### Logistic Regression

### Neural Network

### Decision Tree

In [0]:
import pandas as pd
import pydotplus #pip install pydotplus
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn import metrics



In [0]:
#define Decision Tree
tree = DecisionTreeClassifier(max_depth = 2)
tree.fit(X_test,y_test)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:

y_pred = tree.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8324022346368715


### Preprocess, and fit test data

# Which approach did you choose and why?

In [0]:
# 39.	Join SibSp and Parch columns into a Family column
# 40.	Show the average age of passengers for each passenger class (Hint: use groupby() method)
# 41.	Fill the missing Age and Fare values with the average value


#39
df_test["Family"] = df_test["SibSp"] + df_test["Parch"]
df_test.head()




,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,2


In [0]:
df.head()

,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Family
PassengerId,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",0,22.0,A/5 21171,7.2500,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,PC 17599,71.2833,C,1
3,1,3,"Heikkinen, Miss. Laina",1,26.0,STON/O2. 3101282,7.9250,S,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,113803,53.1000,S,1
5,0,3,"Allen, Mr. William Henry",0,35.0,373450,8.0500,S,0


In [0]:
#40
avg_age = df_test.groupby('Age')
df_test.mean()

Pclass     2.265550
Age       30.272590
SibSp      0.447368
Parch      0.392344
Fare      35.627188
Family     0.839713
dtype: float64

In [0]:
df_test['Age'].value_counts(dropna=False)

NaN     86
24.0    17
21.0    17
22.0    16
30.0    15
        ..
60.5     1
26.5     1
76.0     1
67.0     1
34.5     1
Name: Age, Length: 80, dtype: int64

In [0]:
df_test['Fare'].value_counts(dropna=False)

7.7500     21
26.0000    19
8.0500     17
13.0000    17
7.8958     11
           ..
9.3250      1
14.4583     1
15.0333     1
25.4667     1
21.0750     1
Name: Fare, Length: 170, dtype: int64

In [0]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
Family        0
dtype: int64

In [0]:
df_test['Age'].fillna(value=30.272590, inplace=True)
df_test['Age'].value_counts(dropna=False)

30.27259    86
24.00000    17
21.00000    17
22.00000    16
30.00000    15
            ..
7.00000      1
60.50000     1
26.50000     1
76.00000     1
34.50000     1
Name: Age, Length: 80, dtype: int64

In [0]:
df_test['Fare'].fillna(value=35.627188, inplace=True)
df_test['Fare'].value_counts(dropna=False)

7.7500     21
26.0000    19
8.0500     17
13.0000    17
7.8958     11
           ..
14.4583     1
15.0333     1
25.4667     1
9.2250      1
21.0750     1
Name: Fare, Length: 170, dtype: int64

In [0]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       327
Embarked      0
Family        0
dtype: int64

In [0]:
# 42.	Fill the missing Embarked values with ‘S’ (Southampton port)

In [0]:
df_test['Embarked'].fillna(value='S', inplace=True)
df_test['Embarked'].value_counts(dropna=False)

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [0]:
# 43.	Replace the gender values with 0 for female and 1 for male


In [0]:
df_test['Sex'] = df_test['Sex'].replace(to_replace ="male", 
                 value =1) 
df_test['Sex'] = df_test['Sex'].replace(to_replace ="female", 
                 value =0) 
df_test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",1,34.50000,0,0,330911,7.8292,NaN,Q,0
893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.00000,1,0,363272,7.0000,NaN,S,1
894,2,"Myles, Mr. Thomas Francis",1,62.00000,0,0,240276,9.6875,NaN,Q,0
895,3,"Wirz, Mr. Albert",1,27.00000,0,0,315154,8.6625,NaN,S,0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.00000,1,1,3101298,12.2875,NaN,S,2
...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",1,30.27259,0,0,A.5. 3236,8.0500,NaN,S,0
1306,1,"Oliva y Ocana, Dona. Fermina",0,39.00000,0,0,PC 17758,108.9000,C105,C,0
1307,3,"Saether, Mr. Simon Sivertsen",1,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0


In [0]:
# 44.	Scale the features (Hint: use scale() method)


In [0]:
feature_cols = ['Pclass', 'Sex', 'Age', 'Fare', 'Family']
X = df_test.loc[:, feature_cols]
X.head()



,Pclass,Sex,Age,Fare,Family
PassengerId,,,,,
892,3,1,34.5,7.8292,0
893,3,0,47.0,7.0000,1
894,2,1,62.0,9.6875,0
895,3,1,27.0,8.6625,0
896,3,0,22.0,12.2875,2


In [0]:
from sklearn.preprocessing import scale
X_scaled = scale(X)

In [0]:
# 45.	Make a prediction with a model of your choice